# Hybrid search demo

Please run the `sql_demo.ipynb` notebook first to populate the `scotch_reviews` table. 

Alternatively, look at the bottom of this notebook for steps to manually create the table and take advantage of the new `EMBEDDING()` datatype and function.

Now, let's establish a connection for use in this notebook.

In [1]:
import os, pandas as pd
from sqlalchemy import create_engine, text

username = 'demo'
password = 'demo'
hostname = os.getenv('IRIS_HOSTNAME', 'localhost')
port = '1972' 
namespace = 'USER'
CONNECTION_STRING = f"iris://{username}:{password}@{hostname}:{port}/{namespace}"

engine = create_engine(CONNECTION_STRING)
connection = engine.connect()

## Adding the full text index

Now let's create an iFind (aka [SQL Text Search](https://docs.intersystems.com/irislatest/csp/docbook/DocBook.UI.Page.cls?KEY=GSQLSRCH_txtsrch)) index on our scotch review column using the following command:
```SQL
CREATE INDEX ifind ON scotch_reviews(description) AS %iFind.Index.Basic
```
There's a number of options to refine the behaviour of the [iFind index](https://docs.intersystems.com/irislatest/csp/documatic/%25CSP.Documatic.cls?LIBRARY=%25SYS&CLASSNAME=%25iFind.Index.Basic), as well as more advanced (or basic) types wrt the text indexation, but let's stick with this for now.

In [2]:
res = connection.execute(text("""CREATE INDEX ifind ON scotch_reviews(description) AS %iFind.Index.Basic"""))

Creating an index through DDL will automatically build it, so there's nothing extra to do here.
Now we can query the index using rich fulltext search, including phrase search, wildcard search, fuzzy search, and more (syntax options are described [here](https://docs.intersystems.com/irislatest/csp/documatic/%25CSP.Documatic.cls?LIBRARY=%25SYS&CLASSNAME=%25iFind.Index.Basic)):

```SQL
SELECT name, description FROM scotch_reviews WHERE %ID %FIND search_index(ifind, 'chocolate AND coffee');
SELECT name, description FROM scotch_reviews WHERE %ID %FIND search_index(ifind, 'caramel*');
SELECT name, description FROM scotch_reviews WHERE %ID %FIND search_index(ifind, 'scootish', 3); -- fuzzy search
```

## Creating a Hybrid Search query

Next, we'll need to create a vector for our search string using the same embedding model used for populating the `description_vector` column in our table.

In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2') 
search_vector = model.encode("vanilla", normalize_embeddings=True).tolist()

/opt/anaconda3/envs/iris-vector-search/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm




And now we can start building our hybrid search query:

In [4]:
sql = text("""
    WITH 
    
    filtered AS (
       SELECT %ID AS ID, * FROM scotch_reviews
       WHERE %ID %FIND search_index(ifind, 'vanilla')
    ),
    
    scored AS (
       SELECT name, category, description,
          scotchreviews_ifindrank(ID, 'vanilla') AS IFindScore,
          vector_cosine(description_vector, TO_VECTOR(:search_vec ,FLOAT)) AS VectorScore
       FROM filtered
    ), 
    
    with_rank AS (
       SELECT *,
          RANK() OVER (ORDER BY IFindScore DESC) AS IFindRank,
          RANK() OVER (ORDER BY VectorScore DESC) AS VectorRank
       FROM scored
    ),

    -- using k = 10
    full_score AS (
        SELECT *, (1/(IFindRank + 10) + 1/(VectorRank + 10)) AS Score
        FROM with_rank
    )
    
    SELECT TOP 10 * 
    FROM full_score 
    ORDER BY Score desc""")

# alternatively, you can filter using vector similarity search:
#    filtered AS (
#        SELECT TOP 100 %ID AS ID, * FROM scotch_reviews
#        ORDER BY vector_cosine(description_vector, TO_VECTOR(:search_vec ,FLOAT)) DESC
#    ),

result = connection.execute(sql, { "search_vec": str(search_vector) }).fetchall()
df = pd.DataFrame(result)
pd.set_option('display.max_colwidth', None)  # Easier to read description
df.head()

,name,category,description,IFindScore,VectorScore,IFindRank,VectorRank,Score
0,"Dewar’s 18 year old The Vintage, 40%",Blended Scotch Whisky,"Vanilla laced with spice, fondant icing, grapefruit peel, and lime zest leave the vanilla and floral notes lower down the pecking order. In the mouth, grapefruit and orange dominate the vanilla, yet the mouthfeel is thinner and the acidity tips toward the taste of bitter orange seeds. A lingering bitter orange finish.",0.03321233853724736787,0.468414,2,2,.1666666666666666667
1,"Mortlach, 1997 vintage, 57.1%",Single Malt Scotch,"Matured in a bourbon cask. Thick and creamy, with mouth-coating vanilla, ripe barley, toasted marshmallow, vanilla wafer, key lime pie, golden delicious apple, lemongrass, and hay. The vanilla sweetness lingers to the finish, mixing with dried herbs and hay. I was expecting more from a carefully chosen Mortlach, given its pedigree, but this is still nice. (240 bottles) £250",0.02927189159215022253,0.527266,7,1,.149732620320855615
2,"The Glenrothes, 1987, 43%",Single Malt Scotch,"Amber gold color. Rich aromas of complex fruit and vanilla. Thick and rich in body, with a mouth-coating texture. Flavors of honeyed malt, well structured fruit, and vanilla, with a long finish. \r\n",0.03598003341535131519,0.367166,1,116,.0988455988455988456
3,"Dewar’s 12 year old The Ancestor, 40%",Blended Scotch Whisky,"A straightforward proposition of honey, vanilla sponge cake, barley notes, hints of apple, fresh banana, melon, and bundles of dry straw. It’s a sweetheart: soft vanilla fudge, heather honey, banana-topped banoffee pie, fudge, vanilla sandwich cookies, barley sugar, and lemon peel, with hardly any spice in the early phase. The finish has a snag of pepper at the end, but this is gorgeously tasty, with smooth vanilla fudge all the way.",0.03198225192475672461,0.413522,4,28,.0977443609022556391
4,"Isle of Jura, 16 year old, 43%",Single Malt Scotch,"Antique gold. Gentle aromas of oak, caramel, and a hint of vanilla and sea breeze. Light-medium body, with a creamy texture. Soft, gentle flavors of vanilla, toffee, subtle fruit and brine, with a dryish oaky finish. \r\n",0.03198225192475672461,0.412506,4,31,.095818815331010453



## Creating the table manually

If you're of the more adventurous type, why not create everything from scratch using the most recent `EMBEDDING()` datatype and function ([documented here](https://docs.intersystems.com/irislatest/csp/docbook/DocBook.UI.Page.cls?KEY=GSQL_vecsearch#GSQL_vecsearch_insembed))?
To do so, we'll first need to create an embedding configuration, which we can then refer to when creating our column. Embedding configurations are simple SQL rows in the `%Embedding.Config` table:

```SQL
INSERT INTO %Embedding.Config (Name, EmbeddingClass, Configuration, VectorLength, Description) 
    VALUES ('my-sentence-transformers', '%Embedding.SentenceTransformers', '{"modelName": "all-MiniLM-L6-v2"}', 384, 'SentenceTransformers "all-MiniLM-L6-v2" model')
```

Now create the table and load the data:

```SQL
CREATE TABLE hybrid.scotch_reviews (
    name VARCHAR(255),
    category VARCHAR(255),
    review_point INT,
    price DOUBLE,
    currency VARCHAR(10),
    description VARCHAR(2000),
    description_embedding EMBBEDDING('description', 'my-sentence-transformers')
)

LOAD DATA FROM '~/data/scotch_review.csv'
        COLUMNS (
            id INT,
            name VARCHAR(255),
            category VARCHAR(255),
            review_point INT,
            price DOUBLE,
            currency VARCHAR(10),
            description VARCHAR(2000)
        )
        INTO hybrid.scotch_reviews (name, category, review_point, price, description)
            VALUES (name, category, review_point, price, description)
        USING { "from" : { "file" : { "header" : 1 } } } 
```

Depending on how you mounted this demo, you may need to copy the data file to load from into the container for the `LOAD DATA` command to work:
```Shell
docker cp ./data/scotch_review.csv iris-vector-search-iris-1:/tmp/
```

Now we can create our iFind index, and if you are on 2025.1, you can also add an Approximate Nearest Neighbour index:
```SQL
CREATE INDEX ifind ON hybrid.scotch_reviews(description) AS %iFind.Index.Basic;

-- only on 2025.1!
CREATE INDEX hnsw ON hybrid.scotch_reviews(description_emb) AS HNSW;
```

And now our query becomes (note the small changes in table and embedding column names):
```SQL
WITH 

filtered_text AS (
   SELECT %ID AS ID, * FROM hybrid.scotch_reviews
   WHERE %ID %FIND search_index(ifind, 'vanilla')
),

filtered_vec AS (
    SELECT TOP 100 %ID AS ID, * FROM hybrid.scotch_reviews
    ORDER BY vector_cosine(description_emb, EMBEDDING('vanilla')) DESC
),

scored AS (
   SELECT name, category, description,
      hybrid.scotchreviews_ifindrank(ID, 'vanilla') AS IFindScore,
      vector_cosine(description_emb, EMBEDDING('vanilla')) AS VectorScore
   FROM filtered_text
), 

with_rank AS (
   SELECT *,
      RANK() OVER (ORDER BY IFindScore DESC) AS IFindRank,
      RANK() OVER (ORDER BY VectorScore DESC) AS VectorRank
   FROM scored
),

-- using k = 10
full_score AS (
    SELECT *, (1/(IFindRank + 10) + 1/(VectorRank + 10)) AS Score
    FROM with_rank
)

SELECT TOP 10 * 
FROM full_score 
ORDER BY Score desc
```